In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle

In [2]:
data = pd.read_csv("./Churn_Modelling.csv")

In [3]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
data.drop(['RowNumber' ,'CustomerId', 'Surname' ], axis = 1 , inplace=True)

In [5]:
label_en_gender = LabelEncoder()
data['Gender']= label_en_gender.fit_transform(data['Gender'])

In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder_geo = OneHotEncoder(sparse_output=False)  
geo_encoder = encoder_geo.fit_transform(data[['Geography']]) 
geo_feature_names = encoder_geo.get_feature_names_out(['Geography'])

In [7]:
geo_feature_names

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_df = pd.DataFrame(geo_encoder , columns=geo_feature_names)

In [9]:
data = pd.concat([  data , geo_df] , axis=1)

In [10]:
data.drop(['Geography'] , axis=1 , inplace=True)

In [32]:
with open('label_encoder_gender.pkl' , 'wb') as file :
    pickle.dump(label_en_gender , file)

with open('onehot_encoder_geo.pkl' , 'wb') as file :
    pickle.dump(encoder_geo , file)

In [12]:
X = data.drop(['Exited'] , axis = 1)
y = data['Exited']

In [13]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
with open('scaler.pkl' , 'wb') as file :
    pickle.dump(scaler , file)

## ANN 

In [15]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime


In [16]:
model = Sequential(
    [
        Dense(64 , activation='relu' , input_shape =(X_train.shape[1] ,) ),
        Dense(32 , activation = 'relu'),
        Dense(1 , activation= 'sigmoid')
    ]
)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model.compile(optimizer="adam" , loss="binary_crossentropy" , metrics=['accuracy'])

In [19]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [22]:
tenserboard_callbacks = TensorBoard(log_dir , histogram_freq=1)

In [23]:
early_stop_callback = EarlyStopping(monitor='val_loss' , verbose=True , patience=5 , restore_best_weights=True )

In [24]:
history = model.fit(
    X_train , y_train , validation_data = (X_test , y_test) , epochs = 100 ,
    callbacks =[tenserboard_callbacks , early_stop_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 3ms/step - loss: 0.4645 - accuracy: 0.7959 - val_loss: 0.4055 - val_accuracy: 0.8235
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3986 - accuracy: 0.8313 - val_loss: 0.3719 - val_accuracy: 0.8440
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3651 - accuracy: 0.8515 - val_loss: 0.3557 - val_accuracy: 0.8555
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3474 - accuracy: 0.8574 - val_loss: 0.3471 - val_accuracy: 0.8600
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3414 - accuracy: 0.8605 - val_loss: 0.3449 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3358 - accuracy: 0.8630 - val_loss: 0.3436 - val_accuracy: 0.8550
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3329 - accuracy: 0.8643 - val_loss: 0.3437 - val_accuracy: 0.85

In [25]:
model.save('model.h5')

C:\Users\abhij\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
%tensorboard --logdir logs/fit --port 6007


Reusing TensorBoard on port 6007 (pid 376936), started 0:01:37 ago. (Use '!kill 376936' to kill it.)